<a href="https://colab.research.google.com/github/josh130588/MLAIMAR2024/blob/main/Assignment7_DL_NLP_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Import necessary libraries
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Load the dataset, specifying the encoding
url = 'judge-1377884607_tweet_product_company.csv'
df = pd.read_csv(url, encoding='latin-1') # Try 'latin-1' encoding

# Data Cleaning
def clean_text(text):
    if isinstance(text, str): # Check if the text is a string
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\@\w+|\#', '', text)
        text = re.sub(r'[^A-Za-z\s]', '', text)
        text = text.lower()
        return text
    else:
        return "" # Return an empty string for non-string values

df['cleaned_tweet_text'] = df['tweet_text'].apply(clean_text)

# Text Preprocessing
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_tweet_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_tweet_text'])

max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

label_mapping = {'Positive emotion': 0, 'Negative emotion': 1, 'Neutral emotion': 2, 'No emotion toward brand or product': 3}
df['label'] = df['is_there_an_emotion_directed_at_a_brand_or_product'].map(label_mapping)

# Handle Missing Labels - Drop rows with missing labels
df.dropna(subset=['label'], inplace=True)

# Convert labels to integers after dropping NaNs
df['label'] = df['label'].astype(int)

# Regenerate padded_sequences after dropping rows with missing labels
sequences = tokenizer.texts_to_sequences(df['cleaned_tweet_text']) # Regenerate sequences after dropping rows
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post') # Regenerate padded_sequences

# Train-Test Split after handling missing labels and regenerating padded_sequences
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'], test_size=0.2, random_state=42)

# Model Building
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Model Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print("Model accuracy:", accuracy)

Epoch 1/5
224/224 [==============================] - 23s 80ms/step - loss: 0.8829 - accuracy: 0.5925 - val_loss: 0.8499 - val_accuracy: 0.6124
Epoch 2/5
224/224 [==============================] - 17s 74ms/step - loss: 0.8509 - accuracy: 0.6006 - val_loss: 0.8533 - val_accuracy: 0.6124
Epoch 3/5
224/224 [==============================] - 17s 75ms/step - loss: 0.8502 - accuracy: 0.6006 - val_loss: 0.8487 - val_accuracy: 0.6124
Epoch 4/5
224/224 [==============================] - 17s 76ms/step - loss: 0.8490 - accuracy: 0.6006 - val_loss: 0.8492 - val_accuracy: 0.6124
Epoch 5/5
56/56 [==============================] - 1s 23ms/step - loss: 0.8513 - accuracy: 0.6124
Model accuracy: 0.6124160885810852
